In [ ]:
!pip install face_recognition

     |████████████████████████████████| 100.2MB 71kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=1d90416461bf75b571399c31bed7073c714c24323b21af504d89e1200ea01145
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [ ]:
# Part 1: Face detection from video

import face_recognition
import cv2

# This is a demo of running face recognition on a video file and saving the results to a new video file.
#
# PLEASE NOTE: This example requires OpenCV (the `cv2` library) to be installed only to read from your webcam.
# OpenCV is *not* required to use the face_recognition library. It's only required if you want to run this
# specific demo. If you have trouble installing it, try any of the other demos that don't require it instead.

# Open the input movie file
input_movie = cv2.VideoCapture("test.mp4")
length = int(input_movie.get(cv2.CAP_PROP_FRAME_COUNT))

# Create an output movie file (make sure resolution/frame rate matches input video!)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_movie = cv2.VideoWriter('output-video-10sec.avi', fourcc, 29.97, (1280, 720))

# Load some sample pictures and learn how to recognize them.
lmm_image = face_recognition.load_image_file("rk.jpg")
lmm_face_encoding = face_recognition.face_encodings(lmm_image)[0]

al_image = face_recognition.load_image_file("alex-lacamoire.png")
al_face_encoding = face_recognition.face_encodings(al_image)[0]

known_faces = [
    lmm_face_encoding,
    al_face_encoding
]

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
frame_number = 0

while True:
    # Grab a single frame of video
    ret, frame = input_movie.read()
    frame_number += 1

    # Quit when the input video file ends
    if not ret:
        break

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = frame[:, :, ::-1]

    # Find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        # See if the face is a match for the known face(s)
        match = face_recognition.compare_faces(known_faces, face_encoding, tolerance=0.90)

        # If you had more than 2 faces, you could make this logic a lot prettier
        # but I kept it simple for the demo
        name = None
        if match[0]:
            name = "Ram Kartik"
        elif match[1]:
            name = "Alex Lacamoire"

        face_names.append(name)

    # Label the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        if not name:
            continue

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 25), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)

    # Write the resulting image to the output video file
    print("Writing frame {} / {}".format(frame_number, length))
    output_movie.write(frame)

# All done!
input_movie.release()
cv2.destroyAllWindows()

In [ ]:
# Part 2: Splitting video into frames video

# Importing all necessary libraries
import cv2 
import os

# Read the video from specified path
cam = cv2.VideoCapture("output-video-10sec.avi")

try: 
	
	# creating a folder named data 
	if not os.path.exists('data'): 
		os.makedirs('data') 

# if not created then raise error 
except OSError: 
	print ('Error: Creating directory of data') 

# frame 
currentframe = 0

while(True): 
	
	# reading from frame 
	ret,frame = cam.read() 

	if ret:
		currentframe_str = '0000'+str(currentframe)
		# if video is still left continue creating images 
		name = '/content/data/frame' + str(currentframe_str[-5:]) + '.jpg'
		print ('Creating...' + name) 

		# writing the extracted images 
		cv2.imwrite(name, frame) 

		# increasing counter so that it will 
		# show how many frames are created 
		currentframe += 1
	else: 
		break

# Release all space and windows once done
cam.release() 
cv2.destroyAllWindows()

In [ ]:
# Part 3: Resizing and cropping face in frames

# Code to apply operations on all the images 
# present in a folder one by one 
# operations such as rotating, cropping, 
from PIL import Image 
from PIL import ImageFilter 
import os 

# creating a folder named data 
if not "cropped-frames" in os.listdir():
  os.mkdir("cropped-frames" )

def main(): 
	# path of the folder containing the raw images 
	inPath ="/content/data"

	# path of the folder that will contain the modified image 
	outPath ="/content/cropped-frames"

	for imagePath in os.listdir(inPath): 
		# imagePath contains name of the image 
		inputPath = os.path.join(inPath, imagePath) 

		# inputPath contains the full directory name 
		img = Image.open(inputPath)
		width, height = img.size

		fullOutPath = os.path.join(outPath, 'cropped_'+imagePath) 
		# fullOutPath contains the path of the output 
		# image that needs to be generated 
		img.resize((600, 600), Image.ANTIALIAS).save(fullOutPath)
		img.crop((525, 25, width-300, height-100)) .save(fullOutPath)
		# img.crop((l, t, r, b)) .save(fullOutPath)
    # shutil.move(inPath, 'path_to_save'.format(new_name))

		print(fullOutPath)
# Driver Function 
if __name__ == '__main__': 
    main()

In [ ]:
# Part 4: Findng G channel values in the face

import os
from PIL import Image
import imageio
import numpy as np


dirPath = r"/content/cropped-frames"
dirList = os.listdir(dirPath)
outPath = r"/content/G-values"
face_values=[]

for (dirname, dirs, files) in os.walk(dirPath):
   for filename in sorted(files):
       if filename.endswith('.jpg'):
            print("Opening:"+filename)
            thefile = os.path.join(dirname,filename)
            face = imageio.imread(thefile)
            imageio.imwrite(filename, face[:, :, 1])
            face_gchannel = imageio.imread(filename)
            face_gchannel_arr = np.asarray(face_gchannel)
            face_gmean = np.mean(face_gchannel_arr)
            face_values.append(face_gmean)
            print(face_gmean)

print("Ding!")
print(face_values)
deviations = [face_values[i + 1] - face_values[i] for i in range(len(face_values)-1)] 
print(deviations)

Opening:cropped_frame00000.jpg
129.70911072121157
Opening:cropped_frame00001.jpg
130.06594884107488
Opening:cropped_frame00002.jpg
130.31800535598856
Opening:cropped_frame00003.jpg
130.6594256164004
Opening:cropped_frame00004.jpg
130.8390765536984
Opening:cropped_frame00005.jpg
130.5230621479361
Opening:cropped_frame00006.jpg
129.9179831932773
Opening:cropped_frame00007.jpg
129.69282112845138
Opening:cropped_frame00008.jpg
130.76729891956782
Opening:cropped_frame00009.jpg
129.94314156431804
Opening:cropped_frame00010.jpg
129.91909502262445
Opening:cropped_frame00011.jpg
129.88563671622495
Opening:cropped_frame00012.jpg
130.68675593314248
Opening:cropped_frame00013.jpg
130.32460984393757
Opening:cropped_frame00014.jpg
130.8147308154031
Opening:cropped_frame00015.jpg
130.36698125404007
Opening:cropped_frame00016.jpg
131.11116446578632
Opening:cropped_frame00017.jpg
131.40865823252378
Opening:cropped_frame00018.jpg
131.2301154307877
Opening:cropped_frame00019.jpg
131.6022569027611
Opening

In [ ]:
from google.colab import files
files.download("cropped-frames")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.flush_and_unmount()